# Solution for Homework 1
Solution elaboared by Juan Esteban Grimaldos León

to replicate results on this workbook, set src as root

# Q1 
size of downloaded data
First 1 Download the data:

In [1]:
import pandas as pd

df = pd.read_parquet('homeworks/yellow_tripdata_2023-01.parquet', engine='pyarrow')
print(df.shape)

(3066766, 19)


# Q2
What's the standard deviation of the trips duration in January?

In [2]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [20]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df['duration'].dt.total_seconds() / 60
df['duration'] 

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [23]:
df.duration.std()

42.59435124195458

# Q3
To keep only the records where the duration was between 1 and 60 minutes (inclusive), you can use the following code:


In [25]:
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()
fraction =  (df_filtered.shape[0]/df.shape[0])*100
print(fraction)

98.1220282212598


# Q4

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [6]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [46]:
from sklearn.feature_extraction import DictVectorizer

# Convert the dataframe into a list of dictionaries
train_dicts = df_filtered[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
dv.fit(train_dicts)

# Get the feature matrix
X_train = dv.transform(train_dicts)

# Determine the number of columns in the feature matrix
num_columns = X_train.shape[1]
num_columns

515

In [38]:
train_dicts

[{'PULocationID': 161, 'DOLocationID': 141},
 {'PULocationID': 43, 'DOLocationID': 237},
 {'PULocationID': 48, 'DOLocationID': 238},
 {'PULocationID': 138, 'DOLocationID': 7},
 {'PULocationID': 107, 'DOLocationID': 79},
 {'PULocationID': 161, 'DOLocationID': 137},
 {'PULocationID': 239, 'DOLocationID': 143},
 {'PULocationID': 142, 'DOLocationID': 200},
 {'PULocationID': 164, 'DOLocationID': 236},
 {'PULocationID': 141, 'DOLocationID': 107},
 {'PULocationID': 234, 'DOLocationID': 68},
 {'PULocationID': 79, 'DOLocationID': 264},
 {'PULocationID': 164, 'DOLocationID': 143},
 {'PULocationID': 138, 'DOLocationID': 33},
 {'PULocationID': 33, 'DOLocationID': 61},
 {'PULocationID': 79, 'DOLocationID': 186},
 {'PULocationID': 90, 'DOLocationID': 48},
 {'PULocationID': 113, 'DOLocationID': 255},
 {'PULocationID': 237, 'DOLocationID': 239},
 {'PULocationID': 143, 'DOLocationID': 229},
 {'PULocationID': 137, 'DOLocationID': 79},
 {'PULocationID': 132, 'DOLocationID': 116},
 {'PULocationID': 163, '

In [47]:
y_train = df_filtered.duration.values

# Q5


Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [50]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)


rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

d:\projects\juanes-grimaldos-MLops-datatalks-homework\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261931416412

# Q6

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [53]:
# load the test data

categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [54]:
df_val = read_data('homeworks/yellow_tripdata_2023-02.parquet')

In [55]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts) 

y_pred = lr.predict(X_val)

y_val = df_val.duration.values

mean_squared_error(y_val, y_pred, squared=False)

d:\projects\juanes-grimaldos-MLops-datatalks-homework\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.8118162035401735